We should implement and back-test trading algorithms using historical market data.

The project is divided into three main strategies:

1. Moving Average and Momentum Strategies,
2. Value-Based Strategies, and
3. Sentiment-Based Strategies.

Write our own Back Testing Code

▪ Back Testing simulates how a trading strategy would have performed in the past using historical data.
▪ The purpose is to evaluate the strategy's effectiveness, identify potential issues, and refine it before deploying it in live trading.
▪ Here are key components a back-testing code needs to handle:
    1. Load up and process price and other data
    2. Clean and prepare data
    3. Implement logic to buy and sell based on signals
    4. Define trades and measure their performance over time
    5. Incorporate realistic transaction costs
    6. Calculate metrics – return, drawdown, Sharpe ratio etc.…
    7. Visualise results

Validation and Sanity Checks

▪ Out-of-Sample Testing:
    ▪ After optimizing the strategy on historical data, test it on a separate dataset (out-of-sample data) to verify its robustness.
▪ Sanity Checks:
    ▪ Ensure the backtest is realistic (e.g., no future data leakage, no unrealistic execution assumptions) to prevent overestimating the strategy’s performance.

Strategy 1: Moving Average and Momentum Strategies

Rules based Moving average startegy

Understand & implement moving average strategies
    ▪ Simple Moving Average (SMA)
    ▪ Write code that calculates this to different periods

Tasks:
    1. Implement a strategy where a short-term moving average (e.g., S-day SMA) crosses above or below a long-term moving average (e.g., L-day SMA).
    2. Write code to execute buy orders when the short-term average crosses above the long-term average and sell orders when the opposite occurs.
    3. Test the algorithm on a broad range of stocks (at least 100) from the S&P index for a range of values of S and L
    4. You should report average P&L and variance of P&L for each combination of moving average periods

In [ ]:
import pandas as pd
import numpy as np
import itertools

def calculate_moving_average_crossover_strategy(data, short_window, long_window):
    """
    Calculates the P&L of a moving average crossover strategy.

    Args:
        data (pd.DataFrame): DataFrame with stock price data.
        short_window (int): Short-term moving average window.
        long_window (int): Long-term moving average window.

    Returns:
        float: Total P&L of the strategy.
    """

    signals = pd.DataFrame(index=data.index)
    signals['signal'] = 0.0

    signals['short_mavg'] = data['adjclose'].rolling(window=short_window, min_periods=1).mean()
    signals['long_mavg'] = data['adjclose'].rolling(window=long_window, min_periods=1).mean()

    # Generate signals
    signals.loc[signals.index[short_window:], 'signal'] = np.where(signals['short_mavg'][short_window:] > signals['long_mavg'][short_window:], 1.0, 0.0)

    # Generate trading orders
    signals['positions'] = signals['signal'].diff()

    # Calculate P&L
    positions = signals['positions'].copy()
    positions.iloc[0] = 0
    positions = positions.fillna(0)
    
    daily_returns = data['adjclose'].pct_change()
    pnl = (positions.shift(1) * daily_returns).fillna(0)
    total_pnl = pnl.sum()

    return total_pnl


def backtest_moving_average_strategy(stock_data, short_windows, long_windows):
    """
    Backtests the moving average crossover strategy for a range of stocks and moving average windows.

    Args:
        stock_data (pd.DataFrame): DataFrame with stock price data.
        short_windows (list): List of short-term moving average windows.
        long_windows (list): List of long-term moving average windows.

    Returns:
        pd.DataFrame: DataFrame with average P&L and variance for each combination of moving average periods.
    """

    results = []
    stock_names = stock_data.columns[1:] # Exclude the 'Date' column

    for stock in stock_names:
        stock_results = []
        stock_df = pd.DataFrame({'Date': stock_data['Date'], 'adjclose': stock_data[stock]})
        stock_df['Date'] = pd.to_datetime(stock_df['Date'])
        
        # Drop NaN values only from the 'adjclose' column
        stock_df = stock_df.dropna(subset=['adjclose'])
        
        stock_df = stock_df.set_index('Date')
        
        for short_window, long_window in itertools.product(short_windows, long_windows):
            if short_window >= long_window:
                continue # Skip invalid combinations

            pnl = calculate_moving_average_crossover_strategy(stock_df.copy(), short_window, long_window)
            stock_results.append({'Stock': stock, 'Short Window': short_window, 'Long Window': long_window, 'P&L': pnl})
        
        results.extend(stock_results)

    results_df = pd.DataFrame(results)

    # Calculate average P&L and variance for each combination of moving average periods
    summary_df = results_df.groupby(['Short Window', 'Long Window'])['P&L'].agg(['mean', 'var']).reset_index()
    
    return summary_df


if __name__ == "__main__":
    # Load the stock data
    stock_data = pd.read_csv("stock_data.csv")

    # Define moving average windows to test
    short_windows = [5, 10, 20]
    long_windows = [20, 50, 100, 200]

    # Backtest the strategy
    summary_results = backtest_moving_average_strategy(stock_data, short_windows, long_windows)

    # Print the results
    print(summary_results)

    Short Window  Long Window       mean            var
0              5           20  24.098050  242038.145953
1              5           50  24.008248  242051.791317
2              5          100  24.133380  242045.702099
3              5          200  -0.126066       0.467328
4             10           20  24.211495  242004.945119
5             10           50  23.929910  242062.051448
6             10          100  24.243495  241987.266235
7             10          200  -0.056751       0.175791
8             20           50  24.126270  242119.452688
9             20          100  24.291964  242024.407030
10            20          200  -0.042515       0.361362


Rules based Momentum startegy

Understand the concept of momentum
    ▪ Relative Strength Index (RSI)
    ▪ Write code to calculate these

Tasks:
    1. Implement a strategy that buys assets when momentum indicators signal strength (e.g., RSI > 70) and sells when they signal weakness (e.g., RSI < 30).
    2. Combine momentum signals with moving averages to enhance the strategy.
    3. Test the algorithm on a broad range of stocks (at least 100) from the S&P index
    4. You should report average P&L and variance of P&L for each combination of moving average periods
    5. Once again try five best combinations of RSI you can find both alone and combined with moving averages

ML based Moving average strategy

Tasks:
    1. Build a Deep Learning model that takes in the stock prices and ML indicators and RSI indicators as features
    2. Use a 3-layer neural network (1 hidden layers) where the inputs are the indicators and the output is a buy, sell or hold signal
    3. You train it over a subset of the time series
    4. You test it on another part of the timeseries
    5. See if the changing number of layers or neurons per layer helps
    6. Test the algorithm on a broad range of stocks (at least 100) from the S&P index
    7. You should report average P&L and variance of P&L

ML based Momentum strategy

Tasks:
    1. Implement a strategy that buys assets when momentum indicators signal strength (e.g., RSI > 70) and sells when they signal weakness (e.g., RSI < 30).
    2. Combine momentum signals with moving averages to enhance the strategy.
    3. Once again try five best combinations of RSI you can find both alone and combined with moving averages
    4. Use a machine learning algorithm with price, and various MA values and RSI as features to see if you can predict buy and sell

Strategy 2: Value-Based Strategies

Objectives:
• Understand fundamental metrics such as P/E ratios, book value
• Backtest value-based strategies using historical data.
• Evaluate the performance of the strategies.

Tasks:
    1. Buy stocks with low P/E ratio compared to historical average
    2. The Price-to-Book ratio compares a company's market value to its book value (the net asset value on the balance sheet). A low P/B ratio may indicate that the stock is undervalued relative to its assets.
    3. Use a machine learning algorithm with price, P/E and PtB values as features to see if you can predict buy and sell

Strategy 3: Sentiment-Based Strategies.

▪ Understand and implement sentiment-based trading strategies.
▪ Analyze sentiment data from news articles, social media, and other sources.
▪ Backtest sentiment-based strategies using historical sentiment data.
▪ Evaluate the performance of sentiment-based strategies.

Implement a Sentiment-Based Trading Strategy:
    ▪ Develop a strategy that buys stocks with positive sentiment and sells stocks with negative sentiment.
    ▪ Experiment with different thresholds for sentiment scores to refine the strategy.
    ▪ Combine sentiment analysis with the moving average and value-based strategies from previous sections.
    ▪ Explore how sentiment signals can enhance or detract from other strategies.
    ▪ Use historical sentiment data alongside market data to backtest the sentiment-based strategy.